In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('static/Billionaires Statistics Dataset.csv')
df['selfMade']

0       False
1        True
2        True
3        True
4        True
        ...  
2635     True
2636    False
2637     True
2638     True
2639    False
Name: selfMade, Length: 2640, dtype: bool